In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/keyworded.csv').drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [3]:
df.head()

,pos_tagged,tokens,pos,lemma_tokens,entities,headline,tokens.1,is_sarcastic,keywords
0,"[('We', 'PRP'), ('neither', 'DT'), ('postracia...","['we', 'neither', 'postracial', 'postgender', ...","['PRP', 'DT', 'JJ', 'NN', 'NNP', 'NNP', 'NNP',...","['we', 'neither', 'postracial', 'postgender', ...","[('PERSON', ['Donna', 'Edwards', 'DMd']), ('OR...",dem rep totally nails why congress is falling ...,"['dem', 'rep', 'totally', 'nails', 'why', 'con...",0,"{'edward': 0.42, 'essay': 0.256, 'dmd': 0.216,..."
1,"[('Vegetables', 'NNS'), ('dont', 'VBP'), ('bor...","['vegetables', 'dont', 'boring', 'relegated', ...","['NNS', 'VBP', 'VBG', 'VBN', 'JJ', 'NN', 'IN',...","['vegetable', 'dont', 'bore', 'relegate', 'sid...","[('PERSON', ['Broccoli', 'Chipotle', 'Honey', ...",eat your veggies deliciously different recipes,"['eat', 'your', 'veggies', 'deliciously', 'dif...",0,"{'recipe': 0.365, 'roasted': 0.22, 'cauliflowe..."
2,"[('To', 'TO'), ('extent', 'VB'), ('inheritance...","['to', 'extent', 'inheritance', 'life', 'cumul...","['TO', 'VB', 'NN', 'NNP', 'NN', 'IN', 'NN', 'N...","['to', 'extent', 'inheritance', 'life', 'cumul...","[('ORGANIZATION', ['America', 'Europe', 'Canad...",my white inheritance,"['my', 'white', 'inheritance']",0,"{'anthony': 0.284, 'family': 0.231, 'inheritan..."
3,"[('Even', 'RB'), ('years', 'NNS'), ('experienc...","['even', 'years', 'experience', 'process', 're...","['RB', 'NNS', 'RB', 'RB', 'VBG', 'NNS', 'VBG',...","['even', 'year', 'experience', 'process', 'rev...","[('PERSON', ['Leverage']), ('ORGANIZATION', ['...",ways to file your taxes with less stress,"['ways', 'to', 'file', 'your', 'taxes', 'with'...",0,"{'retirement': 0.492, 'tax': 0.428, 'income': ..."
4,"[('When', 'WRB'), ('Amanda', 'NNP'), ('Peet', ...","['when', 'amanda', 'peet', '’', 'daughter', 'f...","['WRB', 'NNP', 'NNP', 'NNP', 'NN', 'RB', 'VBD'...","['when', 'amanda', 'peet', '’', 'daughter', 'f...","[('PERSON', ['Amanda', 'Peet']), ('PERSON', ['...",amanda peet told her daughter sex is a special...,"['amanda', 'peet', 'told', 'her', 'daughter', ...",0,"{'peet': 0.509, 'frankie': 0.294, 'conan': 0.2..."


In [8]:
df = df.sample(n=10000)

In [9]:
df_X = df[['lemma_tokens']]
df_y = df[['tokens.1']]

In [10]:
for i in df_y.index:
    df_y.at[i, 'tokens.1'] = ['<startseq>'] + eval(df_y['tokens.1'][i]) + ['<endseq>']

## Remove Infrequent Words

### X

In [11]:
word_list = set()
word_sets = [set(eval(l)) for l in df_X['lemma_tokens'].values]
for s in word_sets:
    word_list = word_list.union(s)

In [28]:
print('number of words before:', len(word_list))

number of words before: 87057


In [12]:
word_dic = {i : 0 for i in word_list}

In [15]:
sentences_X = []
for i in range(len(df_X['lemma_tokens'])):
    words_X = eval(df_X['lemma_tokens'].values[i])
    sentences_X.append(words_X)
    for j in range(len(words_X)):
        word_dic[words_X[j]] += 1

In [26]:
word_threshold = 3

new_sentences_X = []
for i, sentence_X in enumerate(sentences_X):
    new_sentences_X.append([word for word in sentence_X if word_dic[word] >= word_threshold])

In [29]:
print('number of words after:', len(set([word for sentence in new_sentences_X for word in sentence])))

number of words after: 33157


In [30]:
sentences_X = new_sentences_X

In [31]:
sum([len(sentence_X) for sentence_X in sentences_X])/len(sentences_X)

245.4086

### y

In [39]:
sentences_y = df_y['tokens.1'].values

In [40]:
word_list = set([word for sentence in sentences_y for word in sentence])

In [41]:
print('number of words before:', len(word_list))

number of words after: 16824


In [44]:
word_dic = {i: 0 for i in word_list}

In [45]:
for sentence_y in sentences_y:
    for word in sentence_y:
        word_dic[word] += 1

In [51]:
word_threshold = 3

new_sentences_y = []
for i, sentence_y in enumerate(sentences_y):
    new_sentences_y.append([word for word in sentence_y if word_dic[word] >= word_threshold])

In [52]:
print('number of words after:', len(set([word for sentence in new_sentences_y for word in sentence])))

number of words after: 5409


In [61]:
sentences_y = new_sentences_y

In [64]:
max([len(sentence_y) for sentence_y in sentences_y])

31

## Save the New Data

In [69]:
df['lemma_tokens'] = sentences_X

In [71]:
df['tokens.1'] = sentences_y

In [73]:
df.to_csv('data/testing.csv')

## Word2Vec

In [54]:
from gensim.models import word2vec

w2v = word2vec.Word2Vec
body_embedding = w2v(sentences_X, min_count=1, size=100)
head_embedding = w2v(sentences_y, min_count=1, size=100)

In [55]:
body_embedding.save('models/body_embedding.model')
head_embedding.save('models/head_embedding.model')

In [12]:
vectors_body = []
vectors_head = []

In [13]:
for i in range(len(sentences_X)):
    vectors_body.append([])
    temp = sentences_X[i]
    for j in range(len(temp)):
        vectors_body[i].append(body_embedding[temp[j]])

D:\Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [14]:
vectors_body[0][5].shape

(256,)

In [15]:
for i in range(len(sentences_y)):
    vectors_head.append([])
    temp = sentences_y[i]
    for j in range(len(temp)):
        vectors_head[i].append(head_embedding[temp[j]])

D:\Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [16]:
vectors_head[0][5].shape

(38,)

## FastText

In [76]:
from gensim.models import FastText

body_embedding_fasttext = FastText(sentences_X, size=100, window=5, min_count=1, workers=4,sg=1)
head_embedding_fasttext = FastText(sentences_y, size=100, window=5, min_count=1, workers=4,sg=1)

In [78]:
body_embedding_fasttext.save('models/body_embedding_fasttext.model')
head_embedding_fasttext.save('models/head_embedding_fasttext.model')